In [1]:
import os
from typing import List, Dict
from athina.steps import Debug, Fn, Chain, PromptExecution, Map, ExtractJsonFromString
from athina.steps.llm import PromptTemplate, PromptMessage
from athina.llms.openai_service import OpenAiService
from athina.keys import OpenAiApiKey

OpenAiApiKey.set_key(os.getenv("OPENAI_API_KEY"))
openai_service = OpenAiService()



/Users/shivsakhuja/dev/athina/athina-evals/.venvtest/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CHAIN 1: 
# - Generate a list of car makes and models
# - Extract the car makes
# - For each car make, generate a tweet using an LLM
def generate_tweet(topic: str) -> str:
    return openai_service.chat_completion(
        messages=[
            {
                "role": "system",
                "content": f"Generate a marketing tweet about {topic}"
            }
        ],
        model="gpt-3.5-turbo")

steps = [
    # Generate a list of cars
    PromptExecution(
        llm_service=openai_service,
        template=PromptTemplate(
            messages=[
                PromptMessage(role="system", content="Generate a list of {items} structured as a JSON array.")
            ]
        ),
        model="gpt-3.5-turbo",
        output_key="cars",
    ),
    ExtractJsonFromString(input_key="cars", output_key="cars"),
    Debug(),
    # Extract the make of the car for each car in the list
    Fn(
        fn=lambda cars: [car['make'] for car in cars],
        input_key="cars",
        output_key="makes"
    ),
    # Generate a marketing tweet for each car make
    Map(
        input_key="makes", 
        fn=generate_tweet,
        output_key="tweets"
        ),
    Debug(),
]

chain = Chain(sequence=steps)
result_chain = chain.run(inputs={
    "items": "4 cars with make, model, and license number."
})

makes = result_chain.get_output("makes")
tweets = result_chain.get_output("tweets")

print("context", result_chain.get_context())
print("makes: ", makes)
print("tweets: ", tweets)

tweets

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


context {'items': '4 cars with make, model, and license number.', 'cars': [{'make': 'Toyota', 'model': 'Camry', 'license_number': 'ABC123'}, {'make': 'Honda', 'model': 'Civic', 'license_number': 'XYZ456'}, {'make': 'Ford', 'model': 'F-150', 'license_number': 'DEF789'}, {'make': 'Chevrolet', 'model': 'Malibu', 'license_number': 'GHI101'}], '__steps__': [{'step': 'PromptExecution', 'output': '```json\n[\n    {\n        "make": "Toyota",\n        "model": "Camry",\n        "license_number": "ABC123"\n    },\n    {\n        "make": "Honda",\n        "model": "Civic",\n        "license_number": "XYZ456"\n    },\n    {\n        "make": "Ford",\n        "model": "F-150",\n        "license_number": "DEF789"\n    },\n    {\n        "make": "Chevrolet",\n        "model": "Malibu",\n        "license_number": "GHI101"\n    }\n]\n```'}, {'step': 'ExtractJsonFromString', 'output': [{'make': 'Toyota', 'model': 'Camry', 'license_number': 'ABC123'}, {'make': 'Honda', 'model': 'Civic', 'license_number':

['"Experience the ultimate in reliability and innovation with Toyota. Elevate your driving experience today! #Toyota #Innovation #Reliability"',
 '"Experience the thrill of the open road with Honda\'s innovative technology and reliable performance. Elevate your driving experience today! #Honda #Innovation #Reliability"',
 '"Unleash the power of the road with Ford - where innovation meets performance for an unforgettable driving experience. #Ford #BuiltFordTough 🚗💨"',
 '"Experience the power and performance of a Chevrolet - where innovation meets style on the open road. #Chevrolet #PowerfulPerformance"']

In [5]:
result_chain.get_context()

{'topic': 'AI',
 'news': [{'title': 'OpenAI has been acquired by Microsoft',
   'content': 'Content 1'},
  {'title': 'Google is launching a new AI research lab in Paris',
   'content': 'Content 2'},
  {'title': 'Meta has just released Llama 4, a multimodal AI model',
   'content': 'Content 3'},
  {'title': 'Apple is investing $1 billion in a new AI research center in Berlin',
   'content': 'Content 4'}],
 '__steps__': [{'step': 'Fn',
   'output': [{'title': 'OpenAI has been acquired by Microsoft',
     'content': 'Content 1'},
    {'title': 'Google is launching a new AI research lab in Paris',
     'content': 'Content 2'},
    {'title': 'Meta has just released Llama 4, a multimodal AI model',
     'content': 'Content 3'},
    {'title': 'Apple is investing $1 billion in a new AI research center in Berlin',
     'content': 'Content 4'}]},
  {'step': 'Debug', 'output': None},
  {'step': 'Map',
   'output': ['OpenAI has been acquired by Microsoft',
    'Google is launching a new AI researc

In [3]:
# CHAIN 2: 
# - Get news from an API. (mocked function)
# - Generate summaries for the news articles.
# - Generate an email combining all the summaries.

def get_news_from_api(topic: str) -> List[Dict[str, str]]:
    return [
        {
            "title": "OpenAI has been acquired by Microsoft",
            "content": "Content 1"
        },
        {
            "title": "Google is launching a new AI research lab in Paris",
            "content": "Content 2"
        },
        {
            "title": "Meta has just released Llama 4, a multimodal AI model",
            "content": "Content 3"
        },
        {
            "title": "Apple is investing $1 billion in a new AI research center in Berlin",
            "content": "Content 4"
        },
    ]

steps = [
    # Get news articles from an API
    Fn(
        fn=get_news_from_api,
        input_key="topic",
        output_key="news"
    ),
    Debug(),
    # Extract the title for each news article
    Map(
        input_key="news",
        fn=lambda news_item: news_item['title'],
        output_key="news_titles"
    ),
    # Convert the news items array into a string
    Fn(
        input_key="news_titles",
        fn=lambda news_titles: "\n".join(news_titles),
        output_key="news_titles_str"
    ),
    Debug(),
    # Generate a summary for each news title
    PromptExecution(
        llm_service=openai_service,
        template=PromptTemplate.simple("Generate a summary for the following news titles: {news_titles_str}. Return a json array. Each element of the array should have these fields: title, summary"),
        model="gpt-3.5-turbo",
        output_key="summaries",
    ),
    # Extract the summaries as a JSON array of objects
    ExtractJsonFromString(input_key="summaries", output_key="summaries_list"),
    # Get the summary string (without the title) for each generated summary
    Map(
        input_key="summaries_list",
        fn=lambda summary: summary['summary'],
        output_key="summaries_without_titles",
    ),
    Debug(),
    # Generate an email combining all the summaries
    PromptExecution(
        llm_service=openai_service,
        template=PromptTemplate.simple("Generate a weekly roundup newsletter email from the following summaries: {summaries_without_titles}."),
        model="gpt-4o",
        output_key="email",
    ),
]

chain = Chain(sequence=steps)
result_chain = chain.run(inputs={ "topic": "AI" })
print("context", result_chain.get_context())

res = result_chain.get_output("email")

res

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


context {'topic': 'AI', 'news': [{'title': 'OpenAI has been acquired by Microsoft', 'content': 'Content 1'}, {'title': 'Google is launching a new AI research lab in Paris', 'content': 'Content 2'}, {'title': 'Meta has just released Llama 4, a multimodal AI model', 'content': 'Content 3'}, {'title': 'Apple is investing $1 billion in a new AI research center in Berlin', 'content': 'Content 4'}], '__steps__': [{'step': 'Fn', 'output': [{'title': 'OpenAI has been acquired by Microsoft', 'content': 'Content 1'}, {'title': 'Google is launching a new AI research lab in Paris', 'content': 'Content 2'}, {'title': 'Meta has just released Llama 4, a multimodal AI model', 'content': 'Content 3'}, {'title': 'Apple is investing $1 billion in a new AI research center in Berlin', 'content': 'Content 4'}]}, {'step': 'Debug', 'output': None}, {'step': 'Map', 'output': ['OpenAI has been acquired by Microsoft', 'Google is launching a new AI research lab in Paris', 'Meta has just released Llama 4, a multim

"Subject: Weekly Tech Roundup: Major AI Developments from Microsoft, Google, Meta, and Apple\n\nDear [Recipient's Name],\n\nWelcome to this week's edition of our Tech Roundup! We've gathered the most exciting news in the world of artificial intelligence to keep you informed about the latest advancements and strategic moves by leading tech giants. Here's what you need to know:\n\n**1. Microsoft Acquires OpenAI**\nMicrosoft has made a significant move to bolster its AI capabilities by acquiring OpenAI, a leading artificial intelligence research lab. This acquisition is expected to enhance Microsoft's research efforts and solidify its position in the AI landscape.\n\n**2. Google Expands AI Research in Paris**\nGoogle is expanding its AI research footprint by opening a new lab in Paris. This initiative underscores Google's ongoing commitment to advancing artificial intelligence technologies and fostering innovation in the field.\n\n**3. Meta Unveils Llama 4**\nMeta, formerly known as Faceb

In [6]:
result_chain.get_context()

{'topic': 'AI',
 'news': [{'title': 'OpenAI has been acquired by Microsoft',
   'content': 'Content 1'},
  {'title': 'Google is launching a new AI research lab in Paris',
   'content': 'Content 2'},
  {'title': 'Meta has just released Llama 4, a multimodal AI model',
   'content': 'Content 3'},
  {'title': 'Apple is investing $1 billion in a new AI research center in Berlin',
   'content': 'Content 4'}],
 '__steps__': [{'step': 'Fn',
   'output': [{'title': 'OpenAI has been acquired by Microsoft',
     'content': 'Content 1'},
    {'title': 'Google is launching a new AI research lab in Paris',
     'content': 'Content 2'},
    {'title': 'Meta has just released Llama 4, a multimodal AI model',
     'content': 'Content 3'},
    {'title': 'Apple is investing $1 billion in a new AI research center in Berlin',
     'content': 'Content 4'}]},
  {'step': 'Debug', 'output': None},
  {'step': 'Map',
   'output': ['OpenAI has been acquired by Microsoft',
    'Google is launching a new AI researc